# Aplicação com um método léxico

Aqui estou mais um dia.

Voltamos para a segunda parte do trabalho! Desta vez, vamos ver como aplicar uma análise de sentimentos com uma biblioteca que não precisa necessariamente de labels, mas que possui uma nota atribuída a todas as palavras. 



## Pré-processamento

Primeiro, vou fazer o pré-processamento necessário. Para essa biblioteca em questão só precisamos remover as tags html. Então precisamos importar as bibliotecas, ler nosso dataframe e retirar as tags. Bora:



In [42]:
#bibliotecas
from bs4 import BeautifulSoup 
import pandas as pd 
import numpy as np 

In [43]:
#lendo o dataframe
df = pd.read_csv('/content/134715_320111_compressed_IMDB Dataset.csv.zip')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [44]:
#removendo as tags 
def remove_tags(texto):
  
  soup = BeautifulSoup(texto)
  texto_sem_tags = soup.get_text()
  return texto_sem_tags

df['review'] = df['review'].apply(remove_tags)

In [45]:
df.head(2) #tudo certinho!

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive


## Aplicação de uma análise de sentimentos com Vader




### Biblioteca vaderSentiment

Para essa parte precisamos primeiro instalar a biblioteca e depois importá-la. Vamos fazer isso:

In [46]:
!pip install vaderSentiment

In [47]:
#importando a classe
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#instanciando
analyser = SentimentIntensityAnalyzer()

### Calculando o compound

Para a análise em si vou criar uma função que passa pelas linhas do dataframe e retorna o compound da função `polarity_score` de cada uma:

In [48]:
#função
def analise(texto):
  for frases in texto:
    df_analisado = analyser.polarity_scores(texto)['compound']
    return df_analisado

In [49]:
#aqui apliquei essa função a coluna review e guardei o resultado dentro de uma nova coluna chamada new_review
df['new_review'] = df['review'].apply(analise)

Vamos ver como ficou nossa nova coluna:

In [50]:
df['new_review'] 

0       -0.9916
1        0.9670
2        0.9519
3       -0.9213
4        0.9744
          ...  
49995    0.9886
49996   -0.6837
49997   -0.9734
49998   -0.8657
49999    0.6975
Name: new_review, Length: 50000, dtype: float64

Deu certo! O único problema é que o valor do compound não retorna labels iguais aos da coluna 'sentiment', e sim valores de 1 a -1. Entretanto, nós sabemos que quanto mais perto de 1 mais positivo é o sentimento, e quanto mais perto de -1 mais negativo. Portanto, vamos criar uma função que transforma esses valores em labels iguais a "positive" e "negative":

In [54]:
#função que transforma valores maiores que 0 em positive e menores em negative
def analise2(score):
  if score > 0:
    return 'positive'
 
  return 'negative'

Vamos aplicar essa função a uma nova coluna chamada 'new_sentiment' e depois visualizar essa coluna:

In [53]:
 #aplicando
 df['new_sentiment'] = df['new_review'].apply(analise2)

In [55]:
#vamos ver como ficou
df['new_sentiment']

0        negative
1        positive
2        positive
3        negative
4        positive
           ...   
49995    positive
49996    negative
49997    negative
49998    negative
49999    positive
Name: new_sentiment, Length: 50000, dtype: object

Agora ta certinho! Com as labels iguais a da coluna 'sentiment' podemos calcular nossas métricas para saber o quão bem nosso modelo previu o sentimento correto. Vamos lá:

### Métricas

Para isso, vou importar a biblioteca `classification_report` das métricas do sklearn:

In [56]:
from sklearn.metrics import classification_report 

In [57]:
print(classification_report(df['sentiment'], df['new_sentiment']))

              precision    recall  f1-score   support

    negative       0.79      0.53      0.64     25000
    positive       0.65      0.86      0.74     25000

    accuracy                           0.70     50000
   macro avg       0.72      0.70      0.69     50000
weighted avg       0.72      0.70      0.69     50000



### Conclusão

Nossa previsão não foi tão boa quanto as dos modelos anteriores... Nossa precision mostrou que que nosso número de falsos positivos não foi bom e ficou maior do que deveria. A recall também não foi tão boa. previmos corretamente só 53% dos sentimentos negativos (muuuuuito pouco) e 86% dos sentimentos positivos, o que é um pouco melhor. A f1 que é a média harmônica da recall não ficou tão boa também. Por fim, nossa acurácia ficou como 70%, não é o ideal mas também não é ruim. 

Nessa parte do trabalho realmente gostei de trabalhar com um modelo que não exigisse labels para treino, nunca tinha feito isso antes! Foi muito bom aprender enquanto fazia. 